In [36]:
import pandas as pd

path_to_data: str = '../../data/raw/raw_extended_data/imagegame_old.jsonl'
save_path: str = '../../data/processed/processed_extended_data/imagegame_old_processed.jsonl'
df = pd.read_json(path_to_data, lines=True)

## Cleanup Info
In ealrier version it was allowed for the model to produce 
/n whats the next instruction within its answer. Now its not

Therefore, in player 1 chats, this must be cleaned up to work for a propper fine-tuning

In [37]:
'''
Cleanup the player 1 chats from image game
'''
def prepare_image_game_chat(data: list):
    pattern: str = "\n"
    clean_data = []
    for i, d in enumerate(data):
        if i % 2 == 0:
            clean_data.append({'role': d['role'], 'content': d['content']})
        else:
            content = d['content'].split(pattern)[0]
            clean_data.append({'role': d['role'], 'content': content})

    return clean_data

In [38]:
df['chat_p1'] = df.chat_p1.apply(lambda x: prepare_image_game_chat(x))

In [39]:
columns_to_keep_p1: list = ['game', 'benchmark_version', 'game_id', 'model', 'experiment', 'episode', 'Aborted', 'Lose', 'Success', 'chat_p1', 'target', 'main_score', 'request_count', 'request_ratio', 'average_expression_tokens']

columns_to_keep_p2: list = ['game', 'benchmark_version', 'game_id', 'model', 'experiment', 'episode', 'Aborted', 'Lose', 'Success', 'chat_p2', 'target', 'main_score', 'request_count', 'request_ratio', 'average_expression_tokens']

df['target'] = df.target_grid

In [40]:
# Create a new DataFrame with duplicated entries
df_player1 = df[columns_to_keep_p1].rename(columns={'chat_p1': 'chat'})
df_player1['player'] = 'player 1'

df_player2 = df[columns_to_keep_p2].rename(columns={'chat_p2': 'chat'})
df_player2['player'] = 'player 2'

# Concatenate both DataFrames
result_df = pd.concat([df_player1, df_player2], ignore_index=True)

# Show the resulting 
len(result_df)

4160

In [41]:
# Define the text to filter out
specific_text = 'INVALID_FORMAT'

# Function to check if specific_text is in any of the dictionaries in the list
def contains_specific_text(row):
    return any(specific_text in d['content'] for d in row)

filtered_data = result_df[~result_df['chat'].apply(contains_specific_text)]

In [42]:
len(result_df), len(filtered_data)

(4160, 4160)

In [34]:
filtered_data.to_json(save_path, orient='records', lines=True)